In [ ]:
! pip install langchain sentence-transformers chromadb pdfplumber openai


In [45]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
import os

In [46]:
loader = PyPDFLoader(r"./cloud_note_2.pdf")
docs = loader.load()
type(docs[1])


langchain_core.documents.base.Document

In [47]:
docs[10]


Document(metadata={'producer': 'LibreOffice 6.4', 'creator': 'Writer', 'creationdate': '2025-08-25T07:27:50+05:30', 'author': 'Sunita', 'source': './cloud_note_2.pdf', 'total_pages': 93, 'page': 10, 'page_label': '11'}, page_content='11\nFor this reason super computing sites and large data centres (large data set) \nmust provide High performance computing (HPC)\nHPC\nSo HPC emphasizes the raw speed performance.\nSpeed was the driving force from scientific, engineering and manufacturing \ncommunity.\nTop 500 high performance computers have been ranked based on floating \npoint speed.\nBut due to network intensiveness, computing is also moving towards High \nThroughput Computing (HTC)\nHTC\nBut high throughput computing is required in high end market oriented \ncomputing systems.\nIt pays more attention towards internet searches and web services.\nSo performance goal is measured and more optimal in HTC.\nLastly not only performance but also it intends towards cost, energy saving, \nsecur

In [48]:
len(docs)

93

In [49]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.split_documents(docs)
len(chunks)
type(chunks[0])

langchain_core.documents.base.Document

In [50]:
len(chunks[1].page_content) ,len(docs[1].page_content)

(977, 3730)

In [51]:
persist_directory = "chroma_store"
hf_model = SentenceTransformer("all-MiniLM-L6-v2")
emb = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [52]:
persist_directory = "chroma_store"  # optional persist
vectordb = Chroma.from_documents(documents=chunks, embedding=emb, persist_directory=persist_directory)
vectordb.persist()


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_8512\3893114834.py:3: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [55]:
hf_model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [57]:
retriever = vectordb.as_retriever(search_kwargs={"k": 4})
query = "What is cloud computing?"
retriever.invoke(query)

[Document(metadata={'creator': 'Writer', 'author': 'Sunita', 'page': 14, 'producer': 'LibreOffice 6.4', 'creationdate': '2025-08-25T07:27:50+05:30', 'source': './cloud_note_2.pdf', 'total_pages': 93, 'page_label': '15'}, page_content='data to the workstations.\n\uf0b7 Trend in IT of moving computing and data from desktops to large\ndata centers.\n\uf0b7 there is on-demand provision of software, hardware, and data as a\nservice.\nDefining a cloud\n\uf0b7 The term cloud is used to refer to different technologies, services, and \nconcepts.\n\uf0b7 It is often associated with:\nvirtualized infrastructure or hardware on demand, utility computing, IT'),
 Document(metadata={'author': 'Sunita', 'source': './cloud_note_2.pdf', 'page_label': '16', 'producer': 'LibreOffice 6.4', 'creator': 'Writer', 'creationdate': '2025-08-25T07:27:50+05:30', 'page': 15, 'total_pages': 93}, page_content='16\noutsourcing, platform as a service and software as a service, other \nthings that now are the focus of th

In [65]:

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY", "")
llm = ChatOpenAI(temperature=0, model="gpt-4o-mini") 

qa_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="map_reduce", retriever=retriever)


question = "what is cloud computing"
resp = qa_chain.invoke(question)
print("=== ANSWER ===")
print(resp)

=== ANSWER ===
{'query': 'what is cloud computing', 'result': 'Cloud computing refers to the delivery of applications and services over the Internet, utilizing a shared pool of configurable computing resources such as networks, servers, storage, applications, and services. It enables on-demand network access to these resources, which can be rapidly provisioned and released with minimal management effort. Cloud computing is characterized by its utility computing model, where resources are available on a pay-per-use basis, allowing users to subscribe to services and configure servers for applications quickly according to demand. It encompasses a wide range of technologies and concepts, influencing how software is built, deployed, and accessed, as well as how IT infrastructure is designed and costs are allocated.'}
